In [1]:
import os
import pandas as pd
import numpy as np
import torch
import random
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import torchvision.transforms.functional as TF
from transformers import AutoImageProcessor, ResNetForImageClassification
import torch
import pandas as pd
import os
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader


In [2]:
import zipfile
import os

zip_file_path = '/kaggle/input/galaxy-zoo-the-galaxy-challenge/images_training_rev1.zip' 
extract_dir = '/kaggle/working/unzipped_images' 

os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"Files extracted to: {extract_dir}")

extracted_files = os.listdir(extract_dir)
print("Extracted files:", extracted_files)


Files extracted to: /kaggle/working/unzipped_images
Extracted files: ['images_training_rev1']


In [3]:
labels_csv='/kaggle/input/galaxy-zoo-the-galaxy-challenge/training_solutions_rev1.zip'
images_folder='/kaggle/working/unzipped_images/images_training_rev1'
labels_df=pd.read_csv(labels_csv)

Now importantly, I went through the paper of galaxy-zoo and found that each image can be classified into multiple subclasses and there is no regularity in it. Basically each class has other classes linked to it too so it wasnt fair to classify it into a single subclass only out of 37. For example, if an image is belonging to class 1 it may also belong to class 3 and 7 further (just an example) and further subclasses (there is diagram given on net that i accessed to get such understanding). Therefore, according my understanding, the subclass 1.1 belonged to elliptical galaxies, subclass 1.2 belonged to lenticular galaxies and subclass 1.3 belonged to ones which dont have any particular shape. This was the broadest classification into galaxies only that has no contradictions with other classes and also all other classes and subclasses are linked to these in some way so it seemed fair to do this. Thus i am doing this classification only in my notebook.

In [4]:
class1_df = labels_df[['GalaxyID', 'Class1.1', 'Class1.2', 'Class1.3']].copy()
class1_df.rename(columns={
    'Class1.1': 'elliptical',
    'Class1.2': 'lenticular',
    'Class1.3': 'other'
}, inplace=True)

class1_df

,GalaxyID,elliptical,lenticular,other
0,100008,0.383147,0.616853,0.000000
1,100023,0.327001,0.663777,0.009222
2,100053,0.765717,0.177352,0.056931
3,100078,0.693377,0.238564,0.068059
4,100090,0.933839,0.000000,0.066161
...,...,...,...,...
61573,999948,0.510379,0.489621,0.000000
61574,999950,0.901216,0.098784,0.000000
61575,999958,0.202841,0.777376,0.019783
61576,999964,0.091000,0.909000,0.000000


In [5]:
def one_hot_encode(row):
    max_value = max(row['elliptical'], row['lenticular'], row['other'])
    if max_value == row['elliptical']:
        return pd.Series({'elliptical': 1, 'lenticular': 0, 'other': 0})
    elif max_value == row['lenticular']:
        return pd.Series({'elliptical': 0, 'lenticular': 1, 'other': 0})
    else:
        return pd.Series({'elliptical': 0, 'lenticular': 0, 'other': 1})

one_hot_df = class1_df[['GalaxyID']].copy()
one_hot_df = one_hot_df.join(class1_df.apply(one_hot_encode, axis=1))

print(one_hot_df.head())

   GalaxyID  elliptical  lenticular  other
0    100008           0           1      0
1    100023           0           1      0
2    100053           1           0      0
3    100078           1           0      0
4    100090           1           0      0


In [6]:

ones_count = one_hot_df[['elliptical', 'lenticular', 'other']].sum()

print(ones_count)


elliptical    26693
lenticular    34826
other            59
dtype: int64


Since the number of other galaxies are very low and almost negligible, i ll focus primarily on the other two only and remove the other column from my dataset

In [7]:

filtered_df = one_hot_df[(one_hot_df['elliptical'] != 0) | (one_hot_df['lenticular'] != 0)]

print(filtered_df)


       GalaxyID  elliptical  lenticular  other
0        100008           0           1      0
1        100023           0           1      0
2        100053           1           0      0
3        100078           1           0      0
4        100090           1           0      0
...         ...         ...         ...    ...
61573    999948           1           0      0
61574    999950           1           0      0
61575    999958           0           1      0
61576    999964           0           1      0
61577    999967           1           0      0

[61519 rows x 4 columns]


In [8]:
label_df = filtered_df.drop(columns=['other'])

label_df['label'] = label_df['elliptical'].apply(lambda x: 1 if x == 1 else 0)

label_df = label_df.drop(columns=['elliptical', 'lenticular'])

In [9]:
label_df

,GalaxyID,label
0,100008,0
1,100023,0
2,100053,1
3,100078,1
4,100090,1
...,...,...
61573,999948,1
61574,999950,1
61575,999958,0
61576,999964,0


Now i access the resnet50 model from the huggingface platform and get it ready for binary classification

In [13]:

processor = AutoImageProcessor.from_pretrained("microsoft/resnet-50")
model = ResNetForImageClassification.from_pretrained("microsoft/resnet-50")

num_classes = 1
in_features = model.classifier[-1].in_features 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.classifier[-1] = nn.Linear(in_features, num_classes)
model.to(device) 


preprocessor_config.json:   0%|          | 0.00/266 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/69.6k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

ResNetForImageClassification(
  (resnet): ResNetModel(
    (embedder): ResNetEmbeddings(
      (embedder): ResNetConvLayer(
        (convolution): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): ReLU()
      )
      (pooler): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (encoder): ResNetEncoder(
      (stages): ModuleList(
        (0): ResNetStage(
          (layers): Sequential(
            (0): ResNetBottleNeckLayer(
              (shortcut): ResNetShortCut(
                (convolution): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
                (normalization): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (layer): Sequential(
                (0): ResNetConvLayer(
                  (convolution): Conv2d(64

In [14]:
torch.save(model.state_dict(),'raw_resnet50.pth')

Preparation of dataset for evalutation and training...note i also included the transformation like random cropping, rotating, contrast and brightness to make the data more diverse.

In [10]:
import os
from PIL import Image
import random
import torch
from torch.utils.data import Dataset
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF

class GalaxyBrightness(object):
    """Custom transform to adjust galaxy brightness."""
    def __init__(self, brightness_range=(0.8, 1.2)):
        self.brightness_range = brightness_range
    
    def __call__(self, img):
        brightness = random.uniform(self.brightness_range[0], self.brightness_range[1])
        return TF.adjust_brightness(img, brightness)

class GalaxyContrast(object):
    """Custom transform to adjust galaxy contrast."""
    def __init__(self, contrast_range=(0.8, 1.2)):
        self.contrast_range = contrast_range
    
    def __call__(self, img):
        contrast = random.uniform(self.contrast_range[0], self.contrast_range[1])
        return TF.adjust_contrast(img, contrast)

class GalaxyZooDataset(Dataset):
    def __init__(self, images_folder, labels_df, train=True):
        self.images_folder = images_folder
        self.labels_df = labels_df 
        self.train = train  

        self.transform_train = transforms.Compose([
            transforms.RandomRotation(360),
            transforms.CenterCrop([256, 256]),
            transforms.Resize([128, 128]),
            GalaxyBrightness(),
            GalaxyContrast(),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
        ])
            
        self.transform_val = transforms.Compose([
            transforms.CenterCrop([256, 256]),
            transforms.Resize([128, 128]),
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
        ])

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        image_id = str(self.labels_df.iloc[idx, 0])  
        label = self.labels_df.iloc[idx, 1] 

        image_path = os.path.join(self.images_folder, f"{image_id}.jpg")
        try:
            image = Image.open(image_path).convert('RGB')  # Convert to RGB
        except FileNotFoundError:
            raise FileNotFoundError(f"Image {image_path} not found.")

        # Apply transformations
        if self.train:
            image = self.transform_train(image)
        else:
            image = self.transform_val(image)

        return image, torch.tensor(label, dtype=torch.float32)  


In [11]:
from sklearn.model_selection import train_test_split

train_df,val_df=train_test_split(label_df,test_size=0.2,random_state=42)
val_dataset = GalaxyZooDataset(images_folder, val_df,train=False)
train_dataset = GalaxyZooDataset(images_folder, train_df,train=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [12]:
train_df.shape

(49215, 2)

In [18]:
import torch
import torch.nn.functional as F
from tqdm import tqdm

def evaluate_model(model, data_loader, device):
    model.eval()  # Set model to evaluation mode
    total_correct = 0
    total_samples = 0
    
    # Use tqdm to display a progress bar
    with torch.no_grad(): 
        for images, labels in tqdm(data_loader, desc="Evaluating", leave=False):
            images, labels = images.to(device), labels.to(device)
            
            # Forward pass
            outputs = model(images)
            
            # Access the logits from the outputs
            logits = outputs.logits if hasattr(outputs, 'logits') else outputs
            
            # Apply sigmoid activation to get probabilities
            probabilities = torch.sigmoid(logits)
            
            # Get predictions by thresholding at 0.5
            predictions = (probabilities >= 0.5).float()  # Convert probabilities to binary predictions
            
            # Since we have only one output node, we compare directly
            correct_labels = labels.view(-1, 1)  # Ensure labels are in the correct shape
            total_correct += (predictions == correct_labels).sum().item()
            total_samples += labels.size(0)

    # Calculate and print overall accuracy
    accuracy = total_correct / total_samples
    print(f"Validation Accuracy: {accuracy * 100:.2f}%")
    
    return accuracy


In [19]:
evaluate_model(model, val_loader, device)

Validation Accuracy: 43.41%


0.4340864759427828

Therefore the accuracy without finetuning reaches 43.41%.

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm

def train_model(model, train_loader, val_loader, device, num_epochs=10, learning_rate=0.001):
    criterion = nn.BCEWithLogitsLoss()  # Use BCE with logits for binary classification
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

    best_val_accuracy = 0  # Track best validation accuracy for model saving

    for epoch in range(num_epochs):
        model.train()  
        total_loss = 0
        total_correct = 0
        total_samples = 0

        with tqdm(total=len(train_loader), desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch") as pbar:
            for batch_idx, (images, labels) in enumerate(train_loader):
                images, labels = images.to(device), labels.float().to(device)
                
                optimizer.zero_grad()
                outputs = model(images)
                
                # Check if outputs contains logits attribute (for models from transformers)
                if hasattr(outputs, 'logits'):
                    logits = outputs.logits.squeeze()  
                else:
                    logits = outputs.squeeze()  # Direct tensor output for torchvision models

                loss = criterion(logits, labels)  # Compute the loss
                loss.backward()
                optimizer.step()

                total_loss += loss.item()
                predictions = torch.sigmoid(logits)  # Apply sigmoid to get probabilities
                predicted_labels = (predictions >= 0.5).float()  # Threshold at 0.5
                
                total_correct += (predicted_labels == labels).sum().item()
                total_samples += labels.size(0)

                # Update progress bar every 5 batches to avoid high frequency of updates
                if batch_idx % 5 == 0:
                    pbar.set_postfix(loss=total_loss / (pbar.n + 1), accuracy=total_correct / total_samples * 100)
                    pbar.update(5)  # Only update every 5 batches
        
        scheduler.step()

        avg_loss = total_loss / len(train_loader)
        avg_accuracy = total_correct / total_samples * 100
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Accuracy: {avg_accuracy:.2f}%")

        # Validation phase
        model.eval()
        total_correct_val = 0
        total_samples_val = 0

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.float().to(device)

                outputs = model(images)
                
                # Again, check if logits are wrapped
                if hasattr(outputs, 'logits'):
                    logits = outputs.logits.squeeze()
                else:
                    logits = outputs.squeeze()

                predictions = torch.sigmoid(logits)  
                predicted_labels = (predictions >= 0.5).float()  

                total_correct_val += (predicted_labels == labels).sum().item()
                total_samples_val += labels.size(0)

        val_accuracy = total_correct_val / total_samples_val * 100
        print(f"Validation Accuracy: {val_accuracy:.2f}%")

        # Save the model if validation accuracy improves
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            torch.save(model.state_dict(), "best_model.pth")
            print(f"Model saved with validation accuracy: {val_accuracy:.2f}%")

# Example usage
# Assuming `train_loader` and `val_loader` are DataLoader objects, and the model is define


In [25]:
train_model(model, train_loader, val_loader, device, num_epochs=4, learning_rate=0.001)

Epoch 1/4: 1540batch [05:28,  4.69batch/s, accuracy=83, loss=0.38]                          


Epoch [1/4], Loss: 0.3803, Accuracy: 82.96%
Validation Accuracy: 84.83%
Model saved with validation accuracy: 84.83%


Epoch 2/4: 1540batch [05:30,  4.66batch/s, accuracy=85.4, loss=0.333]                       


Epoch [2/4], Loss: 0.3330, Accuracy: 85.39%
Validation Accuracy: 83.43%


Epoch 3/4: 1540batch [05:29,  4.68batch/s, accuracy=85.7, loss=0.326]                       


Epoch [3/4], Loss: 0.3263, Accuracy: 85.68%
Validation Accuracy: 83.18%


Epoch 4/4: 1540batch [05:28,  4.68batch/s, accuracy=86.9, loss=0.298]                       


Epoch [4/4], Loss: 0.2981, Accuracy: 86.89%
Validation Accuracy: 86.74%
Model saved with validation accuracy: 86.74%


So the final accuracy for completely fine tuned model of resnet50 comes 86.74%.

In [26]:
torch.save(model.state_dict(), "fine_tuned_resnet50.pth")


Setup of LoRA model, freezing all the upper layers of the base model, and unfreezing of lora layers for training.

In [25]:

class LORALayer(nn.Module):
    def __init__(self, input_dim, output_dim, rank=64):
        super(LORALayer, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.rank = rank

        # LoRA parameters
        self.A = nn.Parameter(torch.randn(input_dim, rank) * 0.01)
        self.B = nn.Parameter(torch.randn(rank, output_dim) * 0.01)

        # Base layer (frozen)
        self.base_layer = nn.Linear(input_dim, output_dim)
        self.base_layer.weight.requires_grad = False
        self.base_layer.bias.requires_grad = False

    def forward(self, x):
        original_output = self.base_layer(x)
        low_rank_update = x @ self.A @ self.B
        return original_output + low_rank_update

processor = AutoImageProcessor.from_pretrained("microsoft/resnet-50")
model = ResNetForImageClassification.from_pretrained("microsoft/resnet-50")

num_features = model.classifier[-1].in_features  # Last layer of classifier

lora_layer = LORALayer(input_dim=num_features, output_dim=1, rank=64).to(device)
model.classifier = nn.Sequential(
    nn.Flatten(),
    lora_layer
).to(device)

# Freeze all parameters except LoRA
for param in model.parameters():
    param.requires_grad = False
for param in lora_layer.parameters():
    param.requires_grad = True


In [26]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

ResNetForImageClassification(
  (resnet): ResNetModel(
    (embedder): ResNetEmbeddings(
      (embedder): ResNetConvLayer(
        (convolution): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): ReLU()
      )
      (pooler): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (encoder): ResNetEncoder(
      (stages): ModuleList(
        (0): ResNetStage(
          (layers): Sequential(
            (0): ResNetBottleNeckLayer(
              (shortcut): ResNetShortCut(
                (convolution): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
                (normalization): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (layer): Sequential(
                (0): ResNetConvLayer(
                  (convolution): Conv2d(64

In [27]:
train_model(model, train_loader, val_loader, device, num_epochs=2, learning_rate=0.0001)

Epoch 1/2: 1540batch [03:57,  6.49batch/s, accuracy=70.1, loss=0.578]                       


Epoch [1/2], Loss: 0.5777, Accuracy: 70.14%
Validation Accuracy: 74.45%
Model saved with validation accuracy: 74.45%


Epoch 2/2: 1540batch [03:54,  6.56batch/s, accuracy=75, loss=0.523]                         


Epoch [2/2], Loss: 0.5230, Accuracy: 75.00%
Validation Accuracy: 75.42%
Model saved with validation accuracy: 75.42%


So the accuracy using LoRA reaches 75.42%.